In [1]:
from bestPlace.bestPlace import location
from bestPlace.bestPlace import helperFunctions
import pandas as pd
import datetime
import numpy as np
import pygsheets
import polyline
#from geojson import Polygon
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from folium.plugins import MarkerCluster
import geopandas as gpd
from sklearn import preprocessing

In [2]:
house_addresses = './tests/test_data_houses.csv'
places_of_importance = './data_places_of_importance.csv'
commuting_locations = './tests/test_data_commuting_stations.csv'
helper = helperFunctions()
location_addresses = helper.read_address_csv(house_addresses)

In [3]:
gc = pygsheets.authorize(client_secret='./client_secret_80237418460-ojb5d5fh62tiup3286i6vaejo715m6l9.apps.googleusercontent.com.json')
sh = gc.open('zillow_properties')
wks = sh[0]
addresses = wks.get_all_records()
location_addresses = pd.DataFrame(addresses)

In [ ]:
house_locations = []
for index, row in location_addresses.iterrows():
    house_data = {'address':row['address'],'state':row['state'],'locationType':row['locationType'],'zip':row['zip']}
    print(house_data)
    house_locations.append(
                            location(house_data,
                                places_of_importance,
                                commuting_locations
                                    )
    )
    

In [5]:
def product_output_df(house):
    average_school_ranking = round(pd.DataFrame(house.school_district_data['rankHistory'])['rankStatewidePercentage'].mean())
    dict_val = pd.DataFrame([{'school_ranking' : average_school_ranking,
                'districtID' : house.school_district_data['districtID'],
                'districtName' : house.school_district_data['districtName']}]).T
    max_duration_from_house = 20
    ind = house.df_travel_distances['duration_x'] < 20
    aa = house.df_travel_distances[ind]
    cc = aa.groupby("destination").head(1)[['destination','total_duration']]
    cc.reset_index()
    travel_df = cc.set_index('destination')
    travel_df = travel_df.reset_index()
    travel_df.set_index('destination')
    travel_df.columns = ['field','values']
    dict_val = dict_val.reset_index()
    dict_val.columns = ['field','values']
    travel_df = travel_df.append(dict_val)
    return(travel_df)



In [32]:
import folium
import pandas as pd
import string
 
NYC_COORDINATES = (41.03, -73.76)

# for speed purposes
MAX_RECORDS = 1000

# create empty map zoomed in on San Francisco
map = folium.Map(location=NYC_COORDINATES, zoom_start=10)
mc = MarkerCluster()
geo_df = gpd.GeoDataFrame()
# add a marker for every record in the filtered data, use a clustered view
for house in house_locations:
    try:
        aaa = product_output_df(house)
        string_val = 'Address:'+house.address + '\n'
        remove = string.punctuation + string.whitespace
        aaa = aaa.reset_index()
        address = house.address
        station = 'Grand Central'
        duration = '88 min'
        html="""
            <h1> This is {address}  </h1><br>
            Distance to 
            <p>
            <code>
                from numpy import *<br>
                exp(-2*pi)
            </code>
            </p>
            """.format(address=house.address)
        html = house.df_travel_aggregate.to_html() + "\n\n" + house.df_travel_aggregate.to_html()
        fields = [' Harlem 125th Street Station, NY','89 E 42nd St, New York, NY 10017']
        aaa['field_replaced'] = aaa['field'].str.replace(' ', '_')

        for field in fields:
            field_stripped = field.replace(' ', '_')
            ggg = str(aaa['field'][aaa['field_replaced'] == field_stripped].values[0])
            ggg2 = str(aaa['values'][aaa['field_replaced'] == field_stripped].values[0])
            string_val = string_val + ggg + '\n' + ggg2 + '\n'
        
        iframe = folium.IFrame(html=html, width=500, height=300)
        popup = folium.Popup(iframe, max_width=2650)
        mc.add_child(folium.Marker(location=[house.address_gps['lat'],house.address_gps['lng']],popup=popup))
        for district in house.school_district_polylines:
            geojson = Polygon(helper.switch_lat_long(district))
            crs = {'init': 'epsg:4326'}
            polygon_df = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[geojson]) 
            polygon_df['school_ranking'] = aaa[aaa['field']=='school_ranking']['values'].values[0]
            polygon_df['districtID'] = aaa[aaa['field']=='districtID']['values'].values[0]
            polygon_df['districtName'] = aaa[aaa['field']=='districtName']['values'].values[0]
            geo_df = geo_df.append(polygon_df)
    except:
        print('failed ',house.address)
map.add_child(mc)  


failed  2736 independence avenue apt4a, bronx
failed  3720 Independence Ave APT 1D, Bronx
failed  1976 Lafontaine Ave APT 1C
failed  3 Fordham Hill Oval APT 7F,bronx
failed  439 E 143rd St, Bronx


In [ ]:
def heatmap_data(house_locations,field):
    data = []
    for house in house_locations:
        try:
            aaa = product_output_df(house)
            value = aaa[aaa['field']==field]['values'].values[0]
            ggg = [house.address,house.address_gps['lat'],house.address_gps['lng'],float(value)]
            data.append(ggg)
        except:
            print('failed' + house.address)
    df = pd.DataFrame(data)
    df.columns = ['address','lat','lng','duration']
    return(df)
        
                                                                           
        
heatmap_data = heatmap_data(house_locations,'89 E 42nd St, New York, NY 10017')



In [170]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
heatmap_data['duration_scaled'] = scaler.fit_transform(heatmap_data['duration'].values.reshape(-1,1))

In [176]:
map_2 = folium.Map(location=NYC_COORDINATES, zoom_start=10)
heatmap = folium.plugins.HeatMap(heatmap_data[['lat','lng','duration_scaled']].values.tolist(),
                                 radius = 10,
                                 max_val=1,
                                 max_zoom=10,
                                 gradient={.4: 'blue', .65: 'lime', 1: 'red'})
map_2.add_child(heatmap)


In [ ]:
vals = geo_df[['districtID','districtName','school_ranking']]
vals.drop_duplicates()

In [95]:
# Add the color for the chloropleth:
folium.Choropleth(
 geo_data=geo_df.to_json(),
 name='choropleth',
 data=vals.drop_duplicates(),
 columns=['districtID','school_ranking'],
 key_on='feature.properties.districtID',
 fill_color='YlGn',
 fill_opacity=0.3,
 line_opacity=0.9,
 legend_name='School (%)'
).add_to(map)
map